# Modelo Condensación electrowetting - Wikramanayake et al.
   

Implementa algunas fórmulas para describir la transferencia de calor en la condensación por gotas, teniendo en cuenta la distribución de tamaños de gotas, basado en los trabajos de Wrikamanayake (2019) y KimKim (2011) 

Calcula la transferencia de calor por medio de de un conjunto de resistencias térmicas y distribución de tamaño de gotas(balance de población) 

### Nomenclatura&nbsp;




**R_dif**=resistencia térmica por la capa de difusión

**T_amb**=Temperatura ambiente [K]

**T_int**=Temperatura de la interfase de la gota [K]

**t_c**=Tasa de condensación por área. Hallada experimentalmente [Kg/m²s]

**h_fg**= Calor latente de condensación [J/kg]

**k_aire**=Conductividad térmica del K_aire [W/m K]

**delta**= grosor de la capa límite [m]

**S_r**=Área proyectada de la gota [m²]

**R_g**: Constante específica de los gases (8.3144598 / masa molar)    J/[Kg K]

**v_g**: Volumen especifico del vapor de agua (1/densidad vapor) [m²/Kg]

**h_fg**: Calor latente de vaporización [J/Kg]

**alfa**=Coeficiente de acomodación o condensación; fracción de moléculas de vapor que se mueven en la gota líquida durante el cambio de fase. Debe ser de entre 0.02 a 0.04 para agua. En Wrikamanayake et al. se usó 0.02

**T_s**=Temperatura de superficie de pared [K]

**T_sat**=Temperatura de saturación [K]

**k1**=Conductividad térmica del agua [W/mk]


**r**=radio de la gota [m]

**theta**=Ángulo de contacto [°]

**delta_coat**=Espesor de la capa recubrimiento [m]

**k_coat**=Conductividad térmica del recubrimiento [W/mK]

**rho**=Densidad del condensado [Kg/m³]

**k_c**=Conductividad térmica del condensado [W/mK]

**sigma**=Tensión superficial [N/m]

**N_s**=número de sitios de nucleación en 10⁹ 1/m² 

**r_min**= radio de gota mínimo [m]

**r_max**= Radio de gota máximo [m]

**c**= Concentración vapor de agua en el aire [mol/m³]

**M_agua**=Peso molecular del agua [Kg/mol]

**D**=Coeficiente de difusión del vapor de agua en el aire [m²/s]

Se hacen las resistencias desde la parte exterior hasta el sustrato

In [1]:
## Importar librerías 

import matplotlib.pyplot as plt
import math
import numpy as np
import scipy
import scipy.integrate as integrate
import sys
import seaborn as sns
from functools import partial


### Parámetros de entrada

In [2]:
T_amb=296.15

T_sat=373
T_s=276.15
T_int=T_s
t_c= 2.7e-05 # Kg/m²s tomado del estado estable para 0V en wrikamaniyake et al
h_fg=2270*1000
k_aire=0.024
R_g=461.522 #Tomado de Sablowski
v_g=(1/0.5937643354220767)
alfa=0.02
theta_0=120

delta_coat=2e-06 #2e-06 CYTOP
k_coat=0.12 # 0.12 CYTOP
k_c=0.58

k1=0.58
rho=997
N_s=2.5e11 # 2.5e11 Tomado de KimKim


HAC=10 #Histéresis de ángulo de contacto
sigma=0.0728 # tensión superficial N/m

##Propiedades para cálculo capa límite

D=0.0000242 #m2/s
P_sat=0.024 #bar @21 C
P_atm=1 # bar
T_boil=373
M_agua=0.018015 #Kg/mol
c=0.68 #mol/m3
humedad=60 #%
R=8.31 # J/molK constante de los gases ideales


#Propiedades eléctricas

voltaje=0 #V

eps_0=8.85418e-12 # F/m
eps_d=2 #2 CYTOP
a_gap=25e-6 # 1/2 del gap entre electrodos [m]
w=50e-6 #ancho electrodos [m]


In [3]:
#Ecuación clásica de Young Lippman-Influencia campo eléctrico en ángulo de contacto 
theta_0=math.radians(theta_0)

def Young_Lippman (theta_0,eps_0,eps_d,w,a_gap,sigma,voltaje):
    capacitancia=(2*eps_0*eps_d*1)*np.log((1+(w/a_gap)+math.sqrt((1+(w/a_gap)**2)-1)))/(math.pi*(50e-6*0.05)) #Wrikamanayake et al
    theta=np.arccos(math.cos(theta_0)+((capacitancia/(2*sigma))*voltaje**2))
    print("cap:",capacitancia)
    return theta

theta=Young_Lippman(theta_0=theta_0,eps_0=eps_0,eps_d=eps_d,w=w,a_gap=a_gap,sigma=sigma,voltaje=voltaje)


print("theta_0:",math.degrees(theta_0),"\n"
      "theta:",math.degrees(theta))

cap: 7.257594244362634e-06
theta_0: 119.99999999999999 
theta: 119.99999999999999


In [4]:
## Determinación grosor capa límite

#D=22.5e-06 * (T_amb/273.15)**(1.8) ## [m²/s] Correlación de "Handbook of tables of applied engineering and science" de Boltz
#x_i=(math.exp((-h_fg/R)*((1/T_int)-(1/T_boil))))
#inf=humedad**(P_sat/P_atm)
#delta= -D*M_agua*c*((math.exp((-h_fg/R)*((1/T_int)-(1/T_boil))))-(humedad**(P_sat/P_atm)))

delta=0.01 # Tomado de Zhao et al (2017)


#Límites de integración

#Radio mínimo

def r_min_KimKim(T_sat, sigma, h_fg, rho, T_amb, T_s):

    r_min = 2*T_sat*sigma / (h_fg * rho * (T_amb-T_s)) 	
    return r_min


theta_r=math.degrees(theta)-(0.5*HAC)
theta_a=math.degrees(theta)+(0.5*HAC)


theta_a, theta_r = [math.radians(x) for x in [theta_a, theta_r]]

const=1 #constante numérica depende de la forma de la gota y la inclinación de la superficie
g=9.81


#Radio máximo

def r_max_KimKim(const, theta_r, theta_a, theta, sigma, rho, g):

    r_max = math.sqrt((6*const*(math.cos(theta_r)-math.cos(theta_a))*math.sin(theta)*sigma)/ \
                      (math.pi*(2-3*math.cos(theta)+(math.cos(theta)**3))*rho*g))
    
    return r_max

r_min=r_min_KimKim(T_sat=T_sat,sigma=sigma,h_fg=h_fg,rho=rho,T_amb=T_amb,T_s=T_s)
r_max=r_max_KimKim(const=const,theta_r=theta_r,theta_a=theta_a,theta=theta,sigma=sigma,rho=rho,g=g)



print("delta:",delta,"\n"
    "r_min:",r_min,"m\n"
     "r_max:",r_max,"m\n"
     "Theta:",theta,"\n")




delta: 0.01 
r_min: 1.1998285605715826e-09 m
r_max: 0.0007420634493637925 m
Theta: 2.0943951023931953 



In [5]:

def R_dif_Wrikamanayake (r,T_amb, T_int, t_c, h_fg):
    
#----------------Resistencia a la difusión del vapor en la capa de no condensables [K/W] ----------------
#---- Se asume que el área superficial de la gota es igual al área proyectada

    R_dif=((T_amb-T_int)/(t_c*h_fg))/(math.pi*r**2)
    return R_dif


def R_cond_aire_Wrikamanayake (r,k_aire, delta):
#--------------Resistencia a la conducción del aire [K/W]---------------------
    R_cond_aire=delta/(k_aire*math.pi*(r**2))
    return R_cond_aire

#--Definición de h_i (coeficiente interfacial)
#h_i=Coeficiente interfacial de transferencia de Calor [W/m2K]
deltaT_sub=T_amb-T_s

h_i=((2*alfa)/(2-alfa))*(1/math.sqrt(2*math.pi*R_g*T_sat))*((h_fg**2)/(v_g*T_s))



def R_int_liq_Wrikamanayake (r, theta, alfa, T_s):

#--------Resistencia de Interfase líquido-gas [K/W]----------------------------

    R_int=1/(h_i*2*math.pi*(r**2)*(1-math.cos(theta)))

    return R_int


def R_cond_gotas_Wrikamanayake(r,theta, k1):
#---------Resistencia a la conducción de las gotas [K/W]----------------------
   
    R_drop=theta/(4*math.pi*k1*r*math.sin(theta))

    return R_drop

def A_mojada(r):
    A_m=math.pi*(r**2)
    return A_m

def Area_T_mojada(r,Area_mojada):
    Area_T_mojada=Area_mojada(r)*n(r)+Area_mojada(r)*N(r)
    return Area_T_mojada
    
    
    
def Q_total_n(r,R_dif2,T_amb,T_s,R_cond_dif,R_int_liq,R_got):
    "Resistencia total por gotas pequeñas"""
    Q_total_n=((T_amb-T_s)/(((1/R_dif2(r)+(1/R_cond_dif(r)))**(-1))+R_int_liq(r)+R_got(r)))*n(r)
    return Q_total_n

    
def Q_total_N(r,R_dif2,T_amb,T_s,R_cond_dif,R_int_liq,R_got):
    "Resistencia total por gotas grandes"
    Q_total_N = ((T_amb-T_s)/(((1/R_dif2(r)+(1/R_cond_dif(r)))**(-1))+R_int_liq(r)+R_got(r)))*N(r)
    return Q_total_N


def N_LeFevre(r, r_max):
    "Distribución de tamaño de gotas para gotas grandes "
    N = 1/(3*math.pi*r**2*r_max) * (r/r_max)**(-2/3)
    return N

    
def r_e_KimKim(N_s):
    """ Radio efectivo de la gota """
    r_e = (4*N_s)**(-0.5)
    return r_e


r_e=r_e_KimKim(N_s)
    
def n_KimKim(r, deltaT_sub, r_min, delta_coat, k_coat, k_c, theta, h_i, rho, h_fg, r_e, r_max):
    """ Distribución de tamaño de gotas para gotas pequeñas """
    A_1 = deltaT_sub / (2*rho*h_fg)
    A_2 = theta * (1-math.cos(theta)) / (4*k_c*math.sin(theta))
    A_3 = 1/(2*h_i) + delta_coat*(1-math.cos(theta)) / (k_coat*(math.sin(theta))**2)
    tau = 3*r_e**2 * (A_2*r_e + A_3)**2 / (A_1*(11*A_2*r_e**2 - 14*A_2*r_e*r_min + 8*A_3*r_e - 11*A_3*r_min))
    B_1 = A_2/(tau*A_1) * ((r_e**2-r**2)/2 + r_min*(r_e-r) - r_min**2*math.log((r-r_min)/(r_e-r_min))) 
    B_2 = A_3/(tau*A_1) * (r_e-r - r_min*math.log((r-r_min)/(r_e-r_min)))
    n = 1/(3*math.pi*r_e**3*r_max) * (r_e/r_max)**(-2/3) * r*(r_e-r_min)/(r-r_min) * (A_2*r+A_3)/(A_2*r_e+A_3) * math.exp(B_1+B_2)
    return n 



# Definir funciones para la tasa de transferencia de calor

R_dif2 = partial(R_dif_Wrikamanayake,T_amb=T_amb, T_int=T_int, t_c=t_c, h_fg=h_fg)
R_dif2.___doc___="Resistencia a la difusión de no condensable"

R_cond_dif = partial(R_cond_aire_Wrikamanayake, k_aire=k_aire, delta=delta)
R_cond_dif.__doc__ = "Resistencia para conducción del aire  en cada gota dependiente del radio r en m"  
   
R_int_liq = partial(R_int_liq_Wrikamanayake, theta=theta, alfa=alfa, T_s=T_s)
R_int_liq.__doc__ = "Resistencia para interfase líquida dependiente del radio en m" 
    
    
R_got= partial(R_cond_gotas_Wrikamanayake,theta=theta, k1=k1)
R_got.__doc__ = "Resistencia conducción en las gotas, dependiente del r en m" 
    
Area_mojada=partial(A_mojada)
Area_mojada.__doc___="Área mojada por la gota"

n = partial(n_KimKim, deltaT_sub=deltaT_sub, r_min=r_min, delta_coat=delta_coat, k_coat=k_coat, k_c=k_c,
                theta=theta, h_i=h_i, rho=rho, h_fg=h_fg, r_e=r_e, r_max=r_max)
n.__doc__ = " Distribución de tamaño de gotas para tamaños pequeños dependiendo del radio r en m"
N = partial(N_LeFevre, r_max=r_max)
N.__doc__ = "drop size distribution for large drops depending on drop radius r in m"

   

"""Integración del flujo de calor de acuerdo a la distribución de gotas"""

A_moj,A_moj_interr= integrate.quad(Area_T_mojada,r_min,r_max,(Area_mojada))
                                   
q_n, q_n_interr = integrate.quad(Q_total_n, r_min, r_max,(R_dif2, T_amb, T_s,R_cond_dif, R_int_liq, R_got))
q_N, q_N_interr = integrate.quad(Q_total_N, r_min, r_max,( R_dif2, T_amb, T_s, R_cond_dif, R_int_liq, R_got))
q = q_n + q_N
    
print ("h_i:", round(h_i,1), "W/m2k\n"
      "r_e:",r_e,"\n"
      "q_N:",q_N,"W/m2\n"
      "q_n:",q_n,"W/m2\n"
      "q:", round(q,2),"W/m2\n"
      "Relacion q_N/q:",round((q_N*100/q),2),"%\n"
       )





h_i: 215216.2 W/m2k
r_e: 1e-06 
q_N: 107.89113050358311 W/m2
q_n: 5.969928074126502 W/m2
q: 113.86 W/m2
Relacion q_N/q: 94.76 %

